In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [19]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from skforecast.exceptions import ResidualsUsageWarning
from skforecast.direct import ForecasterDirect
from skforecast.direct import ForecasterDirectMultiVariate

# Fixtures
from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import y
from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import exog, exog_predict
# from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import series
# from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import exog, exog_predict


transformer_exog = ColumnTransformer(
                       [('scale', StandardScaler(), ['exog_1']),
                        ('onehot', OneHotEncoder(), ['exog_2'])],
                       remainder = 'passthrough',
                       verbose_feature_names_out = False
                   )

In [8]:
forecaster = ForecasterDirect(LinearRegression(), steps=5, lags=3)
forecaster.fit(y=y)
results = forecaster._predict_interval_conformal(
    steps=5, nominal_coverage=0.95, use_in_sample_residuals=True, use_binned_residuals=True
)
results.to_numpy()

array([[ 0.51883519,  0.08813429,  0.94953609],
       [ 0.4584716 ,  0.04326184,  0.87368136],
       [ 0.39962743, -0.04758052,  0.84683537],
       [ 0.40452904, -0.0426789 ,  0.85173698],
       [ 0.41534557, -0.03186238,  0.86255351]])

In [18]:
results

,level,pred,lower_bound,upper_bound
50,l1,0.631143,0.176033,1.086252
51,l1,0.380042,0.128327,0.631757
52,l1,0.332560,0.080845,0.584275


In [19]:
results[['pred', 'lower_bound', 'upper_bound']].to_numpy()

array([[0.63114259, 0.17603311, 1.08625208],
       [0.3800417 , 0.12832655, 0.63175685],
       [0.33255977, 0.08084462, 0.58427492]])

In [3]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from scipy.stats import norm

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import series

In [7]:
forecaster = ForecasterDirectMultiVariate(
                     regressor          = Ridge(random_state=123),
                     level              = 'l1',
                     lags               = 3,
                     steps              = 5,
                     transformer_series = None,
                     binner_kwargs      = {'n_bins': 10}
                 )
    
cv = TimeSeriesFold(
            initial_train_size = len(series) - 20,
            steps              = 5,
            refit              = False
        )

metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
                                            forecaster              = forecaster,
                                            series                  = series,
                                            cv                      = cv,
                                            levels                  = 'l1',
                                            metric                  = 'mean_absolute_error',
                                            add_aggregated_metric   = False,
                                            exog                    = series['l1'].rename('exog_1'),
                                            interval                = 0.9,
                                            interval_method         = 'conformal',
                                            use_in_sample_residuals = True,
                                            use_binned_residuals    = True,
                                            random_state            = 123,
                                            verbose                 = False
                                        )

  0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
metrics_levels.to_numpy()

array([['l1', 0.11916192143057436]], dtype=object)

In [12]:
backtest_predictions[['pred', 'lower_bound', 'upper_bound']].to_numpy()

array([[0.36268205, 0.20433986, 0.52102424],
       [0.51787041, 0.43620381, 0.599537  ],
       [0.52472422, 0.42645324, 0.6229952 ],
       [0.59903598, 0.46654146, 0.7315305 ],
       [0.50738853, 0.42572194, 0.58905513],
       [0.42622724, 0.3313424 , 0.52111209],
       [0.47163992, 0.38997332, 0.55330651],
       [0.70873165, 0.48281375, 0.93464954],
       [0.71565768, 0.48973979, 0.94157557],
       [0.49000773, 0.40834113, 0.57167433],
       [0.61296286, 0.48046834, 0.74545738],
       [0.4031027 , 0.30821786, 0.49798755],
       [0.39728743, 0.30240258, 0.49217227],
       [0.46010284, 0.37843624, 0.54176944],
       [0.63022554, 0.49773102, 0.76272006],
       [0.39528807, 0.30040323, 0.49017292],
       [0.47347775, 0.39181115, 0.55514435],
       [0.76236728, 0.53644939, 0.98828517],
       [0.53407054, 0.43579957, 0.63234152],
       [0.5014202 , 0.4197536 , 0.5830868 ]])

In [13]:
import numpy as np

# Array original 2x2
array_2x2 = np.array([[1, 2, 3], [3, 4, 5]])

# Crear un array 2x2x2 duplicando el original
array_2x2x2 = np.stack([array_2x2, array_2x2], axis=2)
array_2x2x2


array([[[1, 1],
        [2, 2],
        [3, 3]],

       [[3, 3],
        [4, 4],
        [5, 5]]])

In [10]:
array_2x2x2[:,:,0]

array([[1, 2, 3],
       [3, 4, 5]])

In [11]:
array_2x2x2[:,:,1]

array([[1, 2, 3],
       [3, 4, 5]])

In [50]:
levels = ['l1', 'l2', 'l3']
steps = 5
use_binned_residuals = False
nominal_coverage = 0.8

residuals = {
    'l1': np.array([1, 2, 3, 4, 5]),
    'l2': np.array([20, 30, 40, 50, 60]),
    'l3': np.array([300, 400, 500, 600, 700])
}
predictions = np.array([
    [10, 200, 3000],
    [10, 200, 3000],
    [10, 200, 3000],
    [10, 200, 3000],
    [10, 200, 3000]
])

n_levels = len(levels)
correction_factor = np.full(
    shape=(steps, n_levels), fill_value=np.nan, order='C', dtype=float
)
for i, level in enumerate(levels):
    correction_factor[:, i] = np.quantile(
        np.abs(residuals[level]), nominal_coverage
    )

print(correction_factor)

lower_bound = predictions - correction_factor
upper_bound = predictions + correction_factor

predictions_stack = np.array([predictions, lower_bound, upper_bound]).swapaxes(0, 2)

print(predictions_stack)

[[  4.2  52.  620. ]
 [  4.2  52.  620. ]
 [  4.2  52.  620. ]
 [  4.2  52.  620. ]
 [  4.2  52.  620. ]]
[[[  10.     5.8   14.2]
  [  10.     5.8   14.2]
  [  10.     5.8   14.2]
  [  10.     5.8   14.2]
  [  10.     5.8   14.2]]

 [[ 200.   148.   252. ]
  [ 200.   148.   252. ]
  [ 200.   148.   252. ]
  [ 200.   148.   252. ]
  [ 200.   148.   252. ]]

 [[3000.  2380.  3620. ]
  [3000.  2380.  3620. ]
  [3000.  2380.  3620. ]
  [3000.  2380.  3620. ]
  [3000.  2380.  3620. ]]]


In [51]:
for i, level in enumerate(levels):
    print(i, level)
    print(predictions_stack[i, :, :])

0 l1
[[10.   5.8 14.2]
 [10.   5.8 14.2]
 [10.   5.8 14.2]
 [10.   5.8 14.2]
 [10.   5.8 14.2]]
1 l2
[[200. 148. 252.]
 [200. 148. 252.]
 [200. 148. 252.]
 [200. 148. 252.]
 [200. 148. 252.]]
2 l3
[[3000. 2380. 3620.]
 [3000. 2380. 3620.]
 [3000. 2380. 3620.]
 [3000. 2380. 3620.]
 [3000. 2380. 3620.]]


In [25]:
predictions

array([[[  10. ,  200. , 3000. ],
        [  10. ,  200. , 3000. ],
        [  10. ,  200. , 3000. ],
        [  10. ,  200. , 3000. ],
        [  10. ,  200. , 3000. ]],

       [[   5.8,  148. , 2380. ],
        [   5.8,  148. , 2380. ],
        [   5.8,  148. , 2380. ],
        [   5.8,  148. , 2380. ],
        [   5.8,  148. , 2380. ]],

       [[  14.2,  252. , 3620. ],
        [  14.2,  252. , 3620. ],
        [  14.2,  252. , 3620. ],
        [  14.2,  252. , 3620. ],
        [  14.2,  252. , 3620. ]]])

In [24]:
predictions.transpose()

array([[[  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2]],

       [[ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ]],

       [[3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ]]])

In [28]:
np.stack([predictions, lower_bound, upper_bound]).transpose()

array([[[  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2]],

       [[ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ]],

       [[3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ]]])

In [41]:
predictions_stack = np.array([predictions, lower_bound, upper_bound]).swapaxes(0, 2)
predictions_stack

array([[[  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2]],

       [[ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ]],

       [[3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ]]])

In [44]:
for i, level in enumerate(levels):
    print(i, level)
    print(predictions_stack[:, i, :])

0 l1
[[  10.     5.8   14.2]
 [ 200.   148.   252. ]
 [3000.  2380.  3620. ]]
1 l2
[[  10.     5.8   14.2]
 [ 200.   148.   252. ]
 [3000.  2380.  3620. ]]
2 l3
[[  10.     5.8   14.2]
 [ 200.   148.   252. ]
 [3000.  2380.  3620. ]]


In [52]:
predictions_stack

array([[[  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2],
        [  10. ,    5.8,   14.2]],

       [[ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ],
        [ 200. ,  148. ,  252. ]],

       [[3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ],
        [3000. , 2380. , 3620. ]]])

In [43]:
predictions_stack.reshape(-1, 3)

array([[  10. ,    5.8,   14.2],
       [  10. ,    5.8,   14.2],
       [  10. ,    5.8,   14.2],
       [  10. ,    5.8,   14.2],
       [  10. ,    5.8,   14.2],
       [ 200. ,  148. ,  252. ],
       [ 200. ,  148. ,  252. ],
       [ 200. ,  148. ,  252. ],
       [ 200. ,  148. ,  252. ],
       [ 200. ,  148. ,  252. ],
       [3000. , 2380. , 3620. ],
       [3000. , 2380. , 3620. ],
       [3000. , 2380. , 3620. ],
       [3000. , 2380. , 3620. ],
       [3000. , 2380. , 3620. ]])

In [54]:
d = {
    'l1': np.arange(1, 6),
    'l2': np.arange(10, 60, 10),
}

arr = np.array([d[level] for level in ['l1', 'l2']]).T
arr

array([[ 1, 10],
       [ 2, 20],
       [ 3, 30],
       [ 4, 40],
       [ 5, 50]])

In [57]:
np.nanquantile(np.array([1, 2, 3, 4, 5, np.nan]), 0.8)

4.2

In [89]:
# Create a df with 50 cols and 1000 rows numericals
n_cols = 50
n_rows = 10000

np.random.seed(123)
df = pd.DataFrame(np.random.randn(n_rows, n_cols), columns=[f'col_{i}' for i in range(n_cols)])
df.head(3)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49
0,-1.085631,0.997345,0.282978,-1.506295,-0.578600,1.651437,-2.426679,-0.428913,1.265936,-0.866740,...,-0.805367,-1.727669,-0.390900,0.573806,0.338589,-0.011830,2.392365,0.412912,0.978736,2.238143
1,-1.294085,-1.038788,1.743712,-0.798063,0.029683,1.069316,0.890706,1.754886,1.495644,1.069393,...,1.150206,-1.267352,0.181035,1.177862,-0.335011,1.031114,-1.084568,-1.363472,0.379401,-0.379176
2,0.642055,-1.977888,0.712265,2.598304,-0.024626,0.034142,0.179549,-1.861976,0.426147,-1.605410,...,-0.823431,0.130215,1.267299,0.332765,0.556549,-0.212080,0.456271,1.544544,-0.239669,0.143308


In [90]:
my_dict = df.to_dict(orient='list')

In [91]:
levels = df.columns
n_levels = len(levels)
levels

Index(['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7',
       'col_8', 'col_9', 'col_10', 'col_11', 'col_12', 'col_13', 'col_14',
       'col_15', 'col_16', 'col_17', 'col_18', 'col_19', 'col_20', 'col_21',
       'col_22', 'col_23', 'col_24', 'col_25', 'col_26', 'col_27', 'col_28',
       'col_29', 'col_30', 'col_31', 'col_32', 'col_33', 'col_34', 'col_35',
       'col_36', 'col_37', 'col_38', 'col_39', 'col_40', 'col_41', 'col_42',
       'col_43', 'col_44', 'col_45', 'col_46', 'col_47', 'col_48', 'col_49'],
      dtype='object')

In [92]:
my_dict.keys()

dict_keys(['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8', 'col_9', 'col_10', 'col_11', 'col_12', 'col_13', 'col_14', 'col_15', 'col_16', 'col_17', 'col_18', 'col_19', 'col_20', 'col_21', 'col_22', 'col_23', 'col_24', 'col_25', 'col_26', 'col_27', 'col_28', 'col_29', 'col_30', 'col_31', 'col_32', 'col_33', 'col_34', 'col_35', 'col_36', 'col_37', 'col_38', 'col_39', 'col_40', 'col_41', 'col_42', 'col_43', 'col_44', 'col_45', 'col_46', 'col_47', 'col_48', 'col_49'])

In [93]:
%%timeit

for i, level in enumerate(levels):
    np.quantile(
        np.abs(my_dict[level]), nominal_coverage
    )

20.2 ms ± 74.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [94]:
%%timeit

correction_factor = np.full(
    shape=(2, n_levels), fill_value=np.nan, order='C', dtype=float
)
for i, level in enumerate(levels):
    correction_factor[:, i] = np.quantile(
        np.abs(my_dict[level]), nominal_coverage
    )

20.6 ms ± 530 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [95]:
%%timeit

np.quantile(np.abs(df), nominal_coverage, axis=0)

7.78 ms ± 154 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
forecaster = ForecasterDirect(
    LinearRegression(), steps=5, lags=3, differentiation=1
)
forecaster.fit(y=y)
results = forecaster._predict_interval_conformal(
    steps=5, nominal_coverage=0.95, use_in_sample_residuals=True
)

In [21]:
results.to_numpy()

array([[ 0.65659084,  0.17710854,  1.13607314],
       [ 0.6496844 , -0.40541691,  1.7047857 ],
       [ 0.61632235, -0.98830203,  2.22094673],
       [ 0.59545137, -1.53828123,  2.72918398],
       [ 0.64917089, -2.01927988,  3.31762167]])